<a href="https://colab.research.google.com/github/aliennandy13/Stock_Analysis_Heiken_EMA30_10/blob/main/Heinken_EMA_30_10_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import datetime

%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
'''
data_price = pd.read_csv("/content/drive/My Drive/Investing/Day/TCS-EQ_price.csv")
data_H_Day = pd.read_csv("/content/drive/My Drive/Investing/Day/TCS-EQ_Day_Heiken.csv")
data_H_Week = pd.read_csv("/content/drive/My Drive/Investing/Day/TCS-EQ_WK_Heiken.csv")
dates = data_price['Date'].unique()
dates_H_D = data_H_Day['Date'].unique()
dates_H_W = data_H_Week['Date'].unique()
print(dates_H_D)
'''

data_day = pd.read_excel("/content/drive/My Drive/Investing/Day/TCS-EQ_price_HK_EMA.xlsx",dtype=str)
dates_day = data_day['Date'].unique()

data_week = pd.read_excel("/content/drive/My Drive/Investing/Day/TCS-EQ_WK_Heiken.xlsx",dtype=str)
dates_week = data_week['Date'].unique()
#print(dates_day)

In [4]:
def find_Week_Date(d0):
  
  d0 = d0.split('-')
  d0 = list(map(int,d0))
  #print(d0)
  d0 = datetime.datetime(d0[0],d0[1],d0[2])
  dl = (dates_week[0]).split(' ')[0]
  #print(dl)
  dl = dl.split('-')
  dl =  list(map(int,dl))
  #print(dl)
  dl = datetime.datetime(dl[0],dl[1],dl[2])
  
  if d0 >= dl:
    date = (dates_week[0]).split(' ')[0]
  else:
    df = data_week
    row_iterator = df.iterrows()
    _, last = row_iterator.__next__()  # take first item from row_iterator
    
    for i, row in row_iterator:    

        date_curr = (row['Date']).split(' ')[0]
        date_curr = date_curr.split('-')
        date_curr = list(map(int,date_curr))
        d1 = datetime.datetime(date_curr[0], date_curr[1], date_curr[2])
        date_next = (last['Date']).split(' ')[0]
        date_next = date_next.split('-')
        date_next = list(map(int,date_next))
        d2 = datetime.datetime(date_next[0], date_next[1], date_next[2])
        
        if d0 >= d1 and d0 < d2:
          date = (row['Date'])
          #print(row['Date'])
          #print(last['Date'])
          break
        last = row
  return (date)

In [22]:
# Main Logic Layer
# Without SL
buy_flag = False
pnl_overall = []

row_iterator = data_day[::-1].iterrows()
_, last = row_iterator.__next__()

for i, row in row_iterator: 
  #print(row[1]['Date']) 
  if not buy_flag:
    #chk for bullish Candle
    if (last['Hi-O']) == (last['Hi-L']):
      #print(row[1]['Date'])
      if[last['EMA-10']] > [last['EMA-30']]:
        #print(row[1]['Date'])
        day_date = (last['Date']).split(" ")[0]
        #print(day_date)       
        week_date = find_Week_Date(day_date)
        
        df_w = data_week[(data_week['Date'] == week_date)]        
        
        if df_w.iloc[0]['EMA-10'] > df_w.iloc[0]['EMA-30']:
          buy_price = row['TCS-EQ O']
          buy_date = row['Date']
          print("B: " + buy_date + "-------" + buy_price)
          buy_flag = True
  else:
    #chk for next Bearish Candle
    if (last['Hi-O']) == (last['Hi-H']):
      buy_flag = False
      sell_price = row['TCS-EQ O']
      sell_date = row['Date']
      print("S: " + sell_date + "-------" + sell_price)
      pnl =  float(sell_price) -float(buy_price) 
      pnl_overall.append(pnl)
      print("PnL: " + str(pnl))

  last = row

total = sum(pnl_overall)
print(total)

B: 2020-07-29 00:00:00-------2292
S: 2020-08-05 00:00:00-------2260
PnL: -32.0
B: 2020-08-10 00:00:00-------2318
S: 2020-08-26 00:00:00-------2242
PnL: -76.0
B: 2020-09-04 00:00:00-------2275
S: 2020-09-25 00:00:00-------2368
PnL: 93.0
B: 2020-09-30 00:00:00-------2489
S: 2020-10-19 00:00:00-------2774
PnL: 285.0
B: 2020-11-06 00:00:00-------2699
S: 2020-11-14 00:00:00-------2677.95
PnL: -21.050000000000182
B: 2020-11-24 00:00:00-------2734.4
S: 2021-01-01 00:00:00-------2880
PnL: 145.5999999999999
B: 2021-01-05 00:00:00-------3039.6
S: 2021-01-29 00:00:00-------3210.05
PnL: 170.45000000000027
B: 2021-03-24 00:00:00-------3111.2
S: 2021-04-15 00:00:00-------3160.3
PnL: 49.100000000000364
B: 2021-06-09 00:00:00-------3202
S: 2021-06-22 00:00:00-------3304
PnL: 102.0
B: 2021-06-23 00:00:00-------3329
S: 2021-07-05 00:00:00-------3342
PnL: 13.0
729.1000000000004


In [21]:
# Main Logic Layer -2 
# Lets wait till we get the Profit :P
buy_flag = False
pnl_overall = []

row_iterator = data_day[::-1].iterrows()
_, last = row_iterator.__next__()

for i, row in row_iterator: 
  #print(row[1]['Date']) 
  if not buy_flag:
    #chk for bullish Candle
    if (last['Hi-O']) == (last['Hi-L']):
      #print(row[1]['Date'])
      if[last['EMA-10']] > [last['EMA-30']]:
        #print(row[1]['Date'])
        day_date = (last['Date']).split(" ")[0]
        #print(day_date)       
        week_date = find_Week_Date(day_date)        
        df_w = data_week[(data_week['Date'] == week_date)]  
        
        if df_w.iloc[0]['EMA-10'] > df_w.iloc[0]['EMA-30']:
          buy_price = row['TCS-EQ O']
          buy_date = row['Date']
          print("B: " + buy_date + "-------" + buy_price)
          buy_flag = True
  else:
    #chk for next Bearish Candle
    if (last['Hi-O']) == (last['Hi-H']):
      sell_price = row['TCS-EQ O']
      sell_date = row['Date']
      pnl =  float(sell_price) -float(buy_price) 
      
      if pnl > 0:
        pnl_overall.append(pnl)
        print("S: " + sell_date + "-------" + sell_price)
        print("PnL: " + str(pnl))
        buy_flag = False
  last = row

total = sum(pnl_overall)
print(total)

B: 2020-07-29 00:00:00-------2292
S: 2020-09-25 00:00:00-------2368
PnL: 76.0
B: 2020-09-30 00:00:00-------2489
S: 2020-10-19 00:00:00-------2774
PnL: 285.0
B: 2020-11-06 00:00:00-------2699
S: 2021-01-01 00:00:00-------2880
PnL: 181.0
B: 2021-01-05 00:00:00-------3039.6
S: 2021-01-29 00:00:00-------3210.05
PnL: 170.45000000000027
B: 2021-03-24 00:00:00-------3111.2
S: 2021-04-15 00:00:00-------3160.3
PnL: 49.100000000000364
B: 2021-06-09 00:00:00-------3202
S: 2021-06-22 00:00:00-------3304
PnL: 102.0
B: 2021-06-23 00:00:00-------3329
S: 2021-07-05 00:00:00-------3342
PnL: 13.0
876.5500000000006
